In [1]:
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow import models
%matplotlib inline
#https://stackabuse.com/time-series-analysis-with-lstm-using-pythons-keras-library/

In [2]:
df = pd.read_csv("../raw_data/df_energy_weather.csv", index_col=[0])
df = df[['Consommation (MW)','T2M', 'T2MDEW', 'T2MWET', 'TS', 'T2M_RANGE','T2M_MAX', 'T2M_MIN', 'QV2M', 'RH2M', 'PRECTOTCORR', 'PS', 'WS10M',
       'WS50M']]

In [3]:
#df = df.values
len_ = int(0.8*df.shape[0])

# test
len_ = 3397

df_train = df[:len_]
df_test = df[len_-60:]
print(df.shape)
print(df_train.shape)
print(df_test.shape)

(3417, 14)
(3397, 14)
(80, 14)


In [4]:
#df_train = df_train.values
#df_train

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))

df_train = scaler.fit_transform(df_train)
df_train

array([[0.49394453, 0.28495732, 0.48621908, ..., 0.58246347, 0.17412587,
        0.23957754],
       [0.60387826, 0.19841036, 0.32332155, ..., 0.64091858, 0.27342657,
        0.32740411],
       [0.62997864, 0.23079188, 0.29222615, ..., 0.92901879, 0.11118881,
        0.13173986],
       ...,
       [0.08709365, 0.46364439, 0.54452297, ..., 0.56993737, 0.15384615,
        0.1700945 ],
       [0.13986783, 0.49985281, 0.53286219, ..., 0.47181628, 0.14685315,
        0.1606448 ],
       [0.17833028, 0.46335001, 0.60106007, ..., 0.4217119 , 0.42937063,
        0.46081156]])

In [6]:
df.shape[0]

3417

features_set = []
labels = []
for i in range(60, df_train.shape[0]):
    features_set.append(df_train[i-60:i, 0])
    labels.append(df_train[i, 0])

In [7]:
X_train = []
y_train = []
for i in range(60, df_train.shape[0]):
    X_train.append(df_train[i-60:i, 0])
    y_train.append(df_train[i, 0])

In [8]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_train
y_train.shape

(3337,)

In [9]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

(3337, 60, 1)

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

2022-06-16 00:58:07.313429: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/api/keras/optimizers


AlreadyExistsError: Another metric with the same name already exists.

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

es = EarlyStopping(monitor='val_loss', min_delta=0.001, verbose=1, patience=5, restore_best_weights=True) #ajout val_accuracy val_loss

model = models.Sequential()

model.add(layers.LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))

model.add(layers.Dropout(0.2))

model.add(layers.LSTM(units=50, return_sequences=True))
model.add(layers.Dropout(0.2))

model.add(layers.LSTM(units=50, return_sequences=True))
model.add(layers.Dropout(0.2))

model.add(layers.LSTM(units=50))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(units = 1 ,activation = 'linear'))

model.summary()


In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
model.fit(X_train, y_train, epochs = 5, batch_size = 32, callbacks=[es], verbose=1) #ajout de callbacks

In [ ]:
# model.predict(X_test)

In [ ]:
df_test.values

In [ ]:
df_essai = df[len(df) - len(df_train) - 60:].values
df_test = df_test.values


In [ ]:
test_inputs = df_essai.reshape(-1,1)

In [ ]:
#df_test = df_test.reshape(-1,1)

In [ ]:
df_test = scaler.transform(df_test)
df_test

In [ ]:
predictions = model.predict(df_test)

In [ ]:
predictions = scaler.inverse_transform(predictions)